In [1]:
from sympy import mod_inverse

# Redefine the parameters of the elliptic curve and the point P
p = 13
a = -1
b = 1
P = (0, 1)

# Define point addition and point doubling operations on the elliptic curve
# Define addition operation
def elliptic_curve_addition(P, Q, a, p):
    if P == ('inf', 'inf'):
        return Q
    if Q == ('inf', 'inf'):
        return P
    if P[0] == Q[0] and (P[1] != Q[1] or P[1] == 0):
        # P + Q = O (infinity) if they are vertically symmetric or P is a tangent point
        return ('inf', 'inf')
    if P == Q:
        # Point doubling
        lambda_ = (3 * P[0]**2 + a) * mod_inverse(2 * P[1], p) % p
    else:
        # Point addition
        lambda_ = (Q[1] - P[1]) * mod_inverse(Q[0] - P[0], p) % p

    x3 = (lambda_**2 - P[0] - Q[0]) % p
    y3 = (lambda_ * (P[0] - x3) - P[1]) % p

    return (x3, y3)

def ecc_double_and_add(P, k, a, p):
    """Double-And-Add algorithm for scalar multiplication on elliptic curves."""
    result = ('inf', 'inf')  # Infinity
    addend = P

    while k:
        if k & 1:
            result = elliptic_curve_addition(result, addend, a, p)
        addend = elliptic_curve_addition(addend, addend, a, p)
        k >>= 1
    return result

# Test the Double-And-Add algorithm
# Choose a scalar k = 9 for scalar multiplication testing
k = 9
result = ecc_double_and_add(P, k, a, p)

result
# (7, 8)


(7, 8)

In [2]:
# Order of point
def find_order_of_point(P, a, p):
    S = P
    n = 1
    while S != ('inf', 'inf'):  # ('inf', 'inf') 是无穷远点
        S = elliptic_curve_addition(S, P, a, p)
        n += 1
    return n

# calculate order of point
order = find_order_of_point(P, a, p)
print(f"Order of point P: {order}")


Order of point P: 19


In [3]:
# Elliptic Curve secp256k1 Example
# Generate public key from private key using scalar multiplication
from py_ecc.secp256k1 import secp256k1

def generate_public_key(private_key):
    """
    Generate the public key using the secp256k1 elliptic curve from the given private key.
    
    Args:
    private_key (int): The private key, a large integer.
    
    Returns:
    (int, int): The public key, a point on the elliptic curve.
    """
    # Base point of secp256k1
    G = secp256k1.G
    
    # Calculate the public key
    public_key = secp256k1.multiply(G, private_key)
    
    return public_key

# Example: Using a random private key
private_key = 123456789  # In practice, the private key should be a randomly generated large integer, e.g., private_key = int(os.urandom(32).hex(), 16)

# Generate the public key
public_key = generate_public_key(private_key)

# Print the results
print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")

# Output
# Private Key: 123456789
# Public Key: (4051293998585674784991639592782214972820158391371785981004352359465450369227, 88166831356626186178414913298033275054086243781277878360288998796587140930350)


Private Key: 123456789
Public Key: (4051293998585674784991639592782214972820158391371785981004352359465450369227, 88166831356626186178414913298033275054086243781277878360288998796587140930350)
